# Objetivo:

- Carregar os dados do banco das tabelas 'stg_socios_silver' e 'stg_empresas_silver', com o Polars;
- Aplicar as regras de negócio nas tabelas (Quantidade de sócios, Sócios Estrangeiros, doc_alvo);
- Carrega os dados no Postgres na tabela 'prd_empresas_socios_gold'.

### Import das Bibliotecas

In [ ]:
import polars as pl
import os
import psycopg2
import gc
from sqlalchemy import create_engine
from dotenv import load_dotenv
from psycopg2 import sql
from io import StringIO

### Definição dos Diretórios

In [ ]:
DATA_DIR = '../data'

GOLD_DIR = os.path.join(DATA_DIR, 'gold')

# Garante que os diretórios existam
os.makedirs(GOLD_DIR, exist_ok=True)

### Ler as tabelas no Postgres

In [ ]:
# Carrega variáveis de ambiente (.env)
load_dotenv(dotenv_path="../.env")

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [ ]:
# Conexão com o PostgreSQL
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS,
    host=DB_HOST,
    port=DB_PORT
)

### Tabela Empresas

#### Leitura da Raw e Tratamento dos Dados

In [ ]:
# Ler dados em chunks
chunk_size = 500_000  # ajuste conforme memória disponível
first_chunk = True

# Inicializa um DataFrame vazio
chunks = []

with conn.cursor(name="server_side_cursor") as cur:
    cur.itersize = chunk_size
    cur.execute("SELECT * FROM stg_empresas_silver;")
    
    for rows in iter(lambda: cur.fetchmany(chunk_size), []):
        df_chunk = pl.DataFrame(
            rows,
            schema=[desc[0] for desc in cur.description],
            orient="row"
        )
        chunks.append(df_chunk)

# Concatena todos os chunks de uma vez
df_empresas = pl.concat(chunks, rechunk=True)
print(df_empresas.shape)

### Tabela Sócios

#### Leitura Raw e Tratamento de Dados

In [ ]:
# Ler dados em chunks
chunk_size = 500_000  # ajuste conforme memória disponível
first_chunk = True

# Inicializa um DataFrame vazio
chunks = []

with conn.cursor(name="server_side_cursor") as cur:
    cur.itersize = chunk_size
    cur.execute("SELECT * FROM stg_socios_silver;")
    
    for rows in iter(lambda: cur.fetchmany(chunk_size), []):
        df_chunk = pl.DataFrame(
            rows,
            schema=[desc[0] for desc in cur.description],
            orient="row"
        )
        chunks.append(df_chunk)

# Concatena todos os chunks de uma vez
df_socios = pl.concat(chunks, rechunk=True)
print(df_socios.shape)

### Tratamentos

#### Aplicar regra de negócio nos Sócios

In [ ]:
socios_aggs = (
    df_socios
    .group_by("cnpj")
    .agg([
        pl.count("nome_socio").alias("qtde_socios"),
        pl.col("documento_socio")
          .cast(pl.Utf8)
          .str.starts_with("999")
          .max()
          .alias("flag_socio_estrangeiro")
    ])
)

print(socios_aggs.shape)

# --- LIBERAR MEMÓRIA ---
del df_socios
gc.collect()

#### Juntar Empresas com Sócios

In [ ]:
df_gold = (
    df_empresas
    .join(socios_aggs, on="cnpj", how="left")
    .with_columns([
        ((pl.col("cod_porte") == 3) & (pl.col("qtde_socios") > 1)).alias("doc_alvo")
    ])
)

print(df_gold.shape)

# --- LIBERAR MEMÓRIA ---
del df_empresas
del socios_aggs
gc.collect()

### Salvar na Camada Gold

In [ ]:
# Path para salvar o arquivo
csv_empresas_socios_temp = os.path.join(GOLD_DIR, "gold_empresas_socios_temp.csv")

# Se o arquivo já existir, remove antes de começar
if os.path.exists(csv_empresas_socios_temp):
    os.remove(csv_empresas_socios_temp)

# --- Seleciona colunas do df_gold ---
df_gold_filtered = df_gold.select([
    pl.col("cnpj").cast(pl.Utf8),
    pl.col("qtde_socios").cast(pl.Int32),
    pl.col("flag_socio_estrangeiro").cast(pl.Boolean),
    pl.col("doc_alvo").cast(pl.Boolean)
])

# --- LIBERAR MEMÓRIA ---
del df_gold
gc.collect()

# --- Salva CSV com delimitador seguro (~) e quotes apenas quando necessário ---
df_gold_filtered.write_csv(
    file=csv_empresas_socios_temp,
    separator="~",
    quote_style="necessary"
)

print("Arquivo salvo em ", csv_empresas_socios_temp)

### Salvar no Postgres

In [ ]:
# --- Conecta no Postgres ---
cur = conn.cursor()

# --- Cria tabela com schema correto ---
cur.execute(sql.SQL("""
    DROP TABLE IF EXISTS prd_empresas_socios_gold;
    CREATE TABLE prd_empresas_socios_gold (
        cnpj TEXT,
        qtde_socios INT,
        flag_socio_estrangeiro BOOLEAN,
        doc_alvo BOOLEAN
    );
"""))

# --- COPY CSV para o Postgres ---
with open(csv_empresas_socios_temp, "r", encoding="utf-8") as f:
    cur.copy_expert(
        sql.SQL("COPY prd_empresas_socios_gold FROM STDIN WITH CSV HEADER DELIMITER '~'"),
        f
    )

conn.commit()
cur.close()
conn.close()

# --- Remove CSV temporário ---
if os.path.exists(csv_empresas_socios_temp):
    os.remove(csv_empresas_socios_temp)

# --- Limpeza de memória ---
del df_gold_filtered
gc.collect()

print("Tabela 'prd_empresas_socios_gold' criada com sucesso e CSV temporário removido.")